Lets build a model!

In [ ]:
!wget https://download.blender.org/release/Blender2.82/blender-2.82a-linux64.tar.xz blender-2.82a-linux64.tar.xz

In [ ]:
!tar xvf blender-2.82a-linux64.tar.xz


In [ ]:
!cd blender-2.82a-linux64/2.82/python/bin && ./python3.7m -m ensurepip && ./python3.7m pip3 install opencv-python && ./python3.7m pip3 install --upgrade Pillow && ./python3.7m pip3 install --upgrade boto3 


In [ ]:
!sudo docker build --tag odin ./

In [ ]:
!sudo docker stop odin-container

In [6]:

bucket="handtools-mini"
rotation_theta=1
upper_bound=4
rotation_type='X'
start_index=0
import time
timestamp = int(round(time.time() * 1000))


In [ ]:
!echo $timestamp

In [ ]:
!date +"%T"

In [ ]:
!sudo docker run --rm --publish 8080:8080/tcp -e AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY --name odin-container odin $rotation_theta $bucket $rotation_type $upper_bound $timestamp $start_index 
    
    
    

In [ ]:
!date +"%T"

In [ ]:

%%bash
region=us-east-1
sudo $(aws ecr get-login --region ${region} --no-include-email)

image=odin
account=$(aws sts get-caller-identity --query Account --output text)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"
#aws ecr --region ${region} create-repository --repository-name "${image}"
sudo aws ecr batch-delete-image --repository-name $image --image-ids imageTag=latest
sudo docker tag ${image} ${fullname}
sudo docker push ${fullname}


In [48]:
#Generate Step-functions template
#axes=['X','Y','Z']
axes=['X','Y','Z']
cluster='arn:aws:ecs:us-east-1:056149205531:cluster/default'
task_definition='arn:aws:ecs:us-east-1:056149205531:task-definition/Odin:2'
job_name='job_'+str(timestamp)+"_1"
bucket='handtools-mini'
step_function_file_name='step_function_'+str(timestamp)+'.json'
divisions=9
step_size=360/divisions
final_template=''
def generate_step_function_template():
    
    templates=''
    task_template=''
    
    with open('task_template.json', 'r') as file:
        task_template = file.read()
    
    for axis in axes:
        for i in range(0,divisions):
            if i==0:
                start_angle=0
            else:
                start_angle=i*step_size+1
                
            stop_angle=(i+1)*step_size

            #print(start,'-',stop)
            template=task_template.replace('TASK_AXIS', axis)
            template=template.replace('BUCKET', bucket)
            template=template.replace('TASK_DEFINITION', task_definition)
            template=template.replace('JOB_NAME', job_name)
            template=template.replace('CLUSTER_ARN', cluster)
            template=template.replace('START_ANGLE', str(int(start_angle)))
            template=template.replace('STOP_ANGLE', str(int(stop_angle)))

            if(len(templates)>0):
                templates = templates+"," +template
            else:
                templates=template
    step_function_template=''
    with open('step_function_template.json', 'r') as file:
        step_function_template = file.read()
    final_template=step_function_template.replace('INSERT_ECS_TASK_BRANCHES_HERE',templates)

    f = open(step_function_file_name, "w")
    f.write(final_template)
    f.close()
    print('finished writing template: '+step_function_file_name)
generate_step_function_template()    


finished writing template: step_function_1588602572884.json


In [49]:
state_machine_arn='arn:aws:states:us-east-1:056149205531:stateMachine:MyStateMachine'

In [50]:
!aws stepfunctions  update-state-machine --state-machine-arn $state_machine_arn --definition file://$step_function_file_name


{
    "updateDate": 1588612450.382
}


In [51]:
!aws stepfunctions start-execution --state-machine-arn $state_machine_arn

{
    "executionArn": "arn:aws:states:us-east-1:056149205531:execution:MyStateMachine:8a90844f-941c-42ba-8ebf-5279f16fa5d2",
    "startDate": 1588612482.538
}


In [ ]:
!blender-2.82a-linux64/blender --background -b -noaudio -E CYCLES --python euler_engine.py -- 2 A handtools 1

In [ ]:
#To see logs of the container, run following command.
!docker logs odin-container